# Lab. TRIL - Net Zero

Engenharia Computacional para a Emissão Zero no Setor de Óleo e Gás

## Introdução 

Esse é um programa que realiza um processo de leitura, processamento e análise de um conjunto de dados de um reservatório, para encontrar classes de injetividade apresentadas com análise de gráfico.

As funcões a serem desenvolvidas são:
* read_df (recebe inputs csv ou df)
* calc_j (de J para J normalizado)
* proc (KDE e Bayesian Blocks)
* class (encontrar os pontos e classes)
* graf_j (analise dos graficos )

Esse projeto é desenvolvido por:<br>

### Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import warnings

#Serve para ignorar os 'red warnings' que algumas bibliotecas apontam porque tem novas versoes de implementacao
warnings.filterwarnings('ignore') 

## 1 - Preparação de Files 

Essas funções devem receber arquivos csv (em uma formatação pré-estabelecida) que serem lidos, checados e iniciados.<br>
Os arquivos precisam:
* ter colunas nomeadas (de preferência em ordem alfabética)
* não ter valores NAN

In [5]:
'''função de leitura do arquivo csv para df'''

def read_df (arq):
    #leitura pelo pandas
    df = arq
    print(df)        
    #limpeza (valores NAN)
    if any(df.isna()):
        print("df com valores NAN")
        for i in df:
            df = df.fillna(0)
        print("df corrigido, não mais possui valores NAN")
    
    return df

In [ ]:
'''função de checagem de keys em dataframe '''

def checking(df):

    #recebe função de leitura
    df = read_df(df)

    #verificando se falta alguma coluna
    lista_base = ['RQI','pressao','pressao_inversa','distancia','permeabilidade','porosidade']
        
    for i in df.keys(): 
        if lista_base.count(i) == 0:            
            raise KeyError(msg_ERRO = f'variavel {i} não identificada')

    return(True,df)

## 2 - Calculo dos J's

In [ ]:
def normaliza_J (df, *J:list):
    
    #recebendo função de checagem
    df = checking(df)

    #atribuição 
    for j in J:
        
        if J == 'J1':
            J1 = df['RQI']
            df['J1'] = J1
                    
        if J == 'J2':
            J2 = df['RQI']*df['pressao']
            df['J2'] = J2

        if J == 'J3':
            J3 = df['RQI']*df['pressao_inversa']
            df['J3'] = J3
                            
        if J == 'J4':
            J4 = df['RQI'] * df['pressao'] * np.log(df['distancia']) 
            df['J4'] = J4

        if J == 'J5':
            J5 = df['RQI'] * df['pressao_inversa'] * np.log(df['distancia'])
            df['J5'] = J5
        
        if J == 'J6':
            J6 = df['permeabilidade'] * df['porosidade'] * np.log(df['distancia'])
            df['J6'] = J6
        
    #normalização
    for coluna in df.columns[6:]:
        df[f'{coluna}_normalizado'] = (df[coluna] - min(df[coluna])) / (max(df[coluna]) - min(df[coluna]))
    
    print(df.head())
    return df

## 3- Processamento Binning <br> (Bayesian Blocks ou KDE)
<br/>
A classe binning é responsável por processar os dados em KDE ou Bayesian Blocks

In [ ]:
class binning():

    #método construtor
    def __init__(self) -> None:
        pass
    
    def proc_binning(df, *J, binning): #recebe o df e o tipo de binning 

        if binning == 'kde':
            
            '''Bibliotecas'''

            from numpy import array, linspace
            from sklearn.neighbors import KernelDensity
            from scipy.misc import electrocardiogram
            from scipy.signal import argrelmin, find_peaks
            from sklearn.model_selection import GridSearchCV, LeaveOneOut
            import scipy.integrate as integrate
            
            from keras.models import Sequential
            from keras.layers import Dense
            from keras.layers import Dropout
            from keras.wrappers.scikit_learn import KerasClassifier
            from keras.constraints import maxnorm
            from keras.utils import np_utils
            #PARA PICOS
            from scipy.misc import electrocardiogram
            from scipy.signal import find_peaks
            import numpy as np

            #recebendo J normalizados
            df = normaliza_J(df, *J)

            '''Calculo Bandwith''' 

            #encontrar as curvas com bandwidth ideal com validacao cruzada
            for coluna in J:
                X = df[f'{coluna}_normalizado'].values[::].reshape(-1, 1)
                bandwidth = 10 ** np.linspace(0.01, 0.05) 
                grid = GridSearchCV(KernelDensity(), {'bandwidth': bandwidth},cv=LeaveOneOut(len(X)))
                grid.fit(X) 
                ideal_band = grid.best_params_
            
                    
            '''Calculo KDE'''   

            for coluna in J:
                min_i = 0
                max_i = df[f'{coluna}_normalizado'].shape[0]#pega a dimensao da coluna 
                aux = max_i - min_i

                kde= KernelDensity(bandwidth = ideal_band).fit(X)
                dist = np.linspace(0, 1.0, aux) #retorna os pontos em uma distancia equidistante
                log = kde.score_samples(dist.reshape(-1,1)) #calcula a probabilidade logarítmica de cada amostra sob o modelo

                '''Calculo das Particoes''' 

                peaks = find_peaks(log, height=min(log)) # calcula os picos
                valleys = argrelmin(log)[0] #Calcula a minima relativa dos dados
                
                ord = np.argsort(np.abs(np.diff(log[valleys])))#Retorna os indices que classificam o array
                ordValleys_J = np.flip(ord)+1


            
            for coluna in J:

                X = sorted(df[f'{coluna}_normalizado'])                           
                minimo = min(X)
                classes = [minimo]
                for i in range(len(peaks)):
                    classes.append(X[peaks[i]])                        
                classes.append(max(X))

            '''Calculo da Integral KDE''' 

            integral = integrate.trapezoid(np.exp(log),dist)
            if 0.99 < integral < 1.01:
                print(f'A integral é: {integral}')
            else:
                print(f'A integral não está no intervalo correto: {integral}')
                raise KeyError(msg_ERRO = f'A integral precisa estar entre 0.99 e 1.01')
            
               

        '''Calculo Bayesian Blocks''' 

        if binning == 'bb':

            from astropy.stats import bayesian_blocks
            
            resultados = {}

            for coluna in df.columns:
                if coluna in df.columns[12::]:
                    serie = df.query(f"{coluna} > 0")[coluna]
                    resultados[coluna] = [serie, bayesian_blocks(serie)]

        '''Imprimindo Informacoes''' 

        
    #return classes

In [ ]:
#teste de integral
#calculo da integral sobre o score_samples

## 4 - Pickle

In [ ]:
def pickle (df, fileout):
    import pickle as pkl
    


### Graficos dos Binnings

In [ ]:
def graf_binning(df, bins, *J, binning): #receberá o df, a quantidade de bins
                                         #os J e o tipo de binning
    
    if binning == 'kde':
    elif binning == 'bb':

## 5 - Análise dos Gráficos

In [ ]:
class graficos:

    def grafico_kde(df, coluna):
        


    def grafico_bb(df, bins_bb, coluna):
        #modificar labels para for
        labels = {
            "J1_normalizado" : "RQI",
            "J2_normalizado" : "RQI * Pressao",
            "J3_normalizado" : "RQI * Pressao Inversa",
            "J4_normalizado" : "RQI * Pressao * ln(distancia)",
            "J5_normalizado" : "RQI * Pressao Inversa * ln(distancia)", 
            "J6_normalizado" : "Permeabilidade * Porosidade * ln(distancia)",
            "J7_normalizado" : "ln(Permeabilidade) * ln(Porosidade) * ln(distancia)"
        }
        
        ax = plt.figure(figsize=(20, 12))
        ax = plt.title(f"Histograma '{coluna} = {labels[coluna]}' utilizando blocos bayesianos", fontsize = 24)
        ax = plt.xlabel("X", fontsize = 18)
        ax = plt.ylabel("Y", fontsize = 18)
        ax = plt.hist(df, bins = bins_bb, color='g')
        ax = plt.grid(True)
        plt.savefig(f'./dados/Analise de Js/{coluna}.jpeg', format='jpeg')
        plt.show(ax)

## 6 - Menu

Aqui vamos chamar o pipeline e apresentar opções ao usuário chamando as funções

In [ ]:
print(' TRIL - NetZero '.center(30,'*'))
print(' Bem Vindo '.center(30,'*'))

In [3]:
#arq = str(print("Insira o caminho para o arquivo: ")) #recebemos o csv

arq = pd.read_csv("./dados/variaveis.csv")

In [7]:
#chamando a funçao de preparação
a = read_df(arq)

            RQI  pressao_inversa  pressao  porosidade  permeabilidade  \
0      0.112898           4543.9   4608.2    0.062768         0.81143   
1      0.085400           4548.2   4604.2    0.050000         0.36985   
2      0.088106           4552.9   4600.0    0.050000         0.39366   
3      0.086471           4557.9   4595.6    0.050000         0.37919   
4      0.088106           4563.0   4590.9    0.050000         0.39366   
...         ...              ...      ...         ...             ...   
85927  0.064116           4597.3   4560.3    0.112380         0.46856   
85928  0.065263           4602.9   4555.2    0.108470         0.46856   
85929  0.066356           4608.3   4550.4    0.104920         0.46856   
85930  0.039787           4613.5   4545.7    0.107880         0.17321   
85931  0.038431           4618.7   4541.1    0.115630         0.17321   

       distancia  
0            0.0  
1            0.0  
2            0.0  
3            0.0  
4            0.0  
...      

In [6]:
df = arq
for i in df:
        print(i)

RQI
pressao_inversa
pressao
porosidade
permeabilidade
distancia


### opções

In [ ]:
#o df precisa passar pelo calculo dos J's para prosseguir
#o usuário deve ter a opção de escolher quais calculos fazer? sim, quais J' normalizados

In [ ]:
opcao = 0
while opcao != 3:
    print('''   Qual tipo de processamento voce deseja executar?
            digite [1] para Bayesian Blocks
            digite [2] para KDE
            digite [3] para ambos
            digite [4] para voltar ao passo anterior ''')
    opcao = int(input())

    #if opcao == 1:
        # função BB
        #print("Processo de Binning por Bayesian Block completado")
        #print(imagens dos graficos de bayesian blocks para cada J normalizado)
        #o programa pode salvar essas imagens diretamente em um arquivo

    #elif opcao == 2:
        #função KDE
        #print("Processo de Binning por KDE completado")
        #print(imagens dos graficos de KDE para cada J normalizado)
        #o programa pode salvar essas imagens diretamente em um arquivo

    #elif opcao == 3:
        # função BB
        #função KDE
    
    #elif opcao == 4:
        #break #volta do começo

    #else:
        #print("Esse e um caractere invalido. Deseja recomeçar? Digite [0]")
        #opcao = 0

In [ ]:
print("Digite se deseja realizar a classificacao: s/n")
resposta = input()

#if resposta == 's':
    #retornar funcao de classificacao

#if resposta == 'n':
    #print("Fim do Programa")
else: 
    

In [ ]:
bandwidth = 0.01